# Winter Wheat BBCH Prediction Model using Growing Degree Days (GDD)

This notebook builds a phenological prediction model for winter wheat (*Triticum aestivum*) using:
- **DWD Phenology Data**: Historical BBCH stage observations from German weather stations
- **DWD Climate Data**: Daily temperature data for calculating Growing Degree Days

## BBCH Stages for Winter Wheat
The BBCH scale is a standardized phenological development scale:
- **BBCH 10**: Emergence (first leaf through soil surface)
- **BBCH 21**: Beginning of tillering
- **BBCH 31**: Beginning of stem elongation
- **BBCH 51**: Beginning of heading
- **BBCH 61**: Beginning of flowering
- **BBCH 75**: Medium milk ripening
- **BBCH 87**: Hard dough (end of ripening)

## Growing Degree Days (GDD)
GDD accumulates thermal time above a base temperature:
$$GDD = \sum_{i=1}^{n} max(0, T_{mean,i} - T_{base})$$

For winter wheat, typical base temperature is **0°C** (some studies use 5°C).

## 1. Setup and Parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import requests
import zipfile
import io
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries loaded successfully!")

In [ ]:
# Model Parameters
PARAMS = {
    # Growing Degree Days parameters
    'T_base': 0.0,           # Base temperature for GDD calculation (°C)
    'T_upper': 30.0,         # Upper temperature cutoff (°C)
    
    # Growing season definition
    'season_start_month': 1,  # January (winter wheat resumes growth)
    'season_start_day': 1,
    
    # BBCH stages tracked by DWD for winter wheat
    'bbch_stages': {
        10: 'Emergence',
        12: 'Two leaves unfolded',
        21: 'Beginning of tillering',
        31: 'Beginning of stem elongation',
        41: 'Flag leaf sheath extending',
        51: 'Beginning of heading',
        61: 'Beginning of flowering',
        75: 'Medium milk ripening',
        87: 'Hard dough'
    },
    
    # Data sources
    'phenology_url_hist': 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/phenology/annual_reporters/crops/historical/PH_Jahresmelder_Landwirtschaft_Kulturpflanze_Winterweizen_1925_2023_hist.txt',
    'phenology_url_recent': 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/phenology/annual_reporters/crops/recent/PH_Jahresmelder_Landwirtschaft_Winterweizen_akt.txt',
    'climate_base_url': 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/',
    'stations_url': 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/KL_Tageswerte_Beschreibung_Stationen.txt'
}

print("Parameters defined:")
print(f"  Base temperature: {PARAMS['T_base']}°C")
print(f"  Upper temperature cutoff: {PARAMS['T_upper']}°C")
print(f"  BBCH stages tracked: {list(PARAMS['bbch_stages'].keys())}")

## 2. Download DWD Data

In [ ]:
# Set up data directory relative to notebook location
DATA_DIR = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'data')
os.makedirs(DATA_DIR, exist_ok=True)
print(f"Data directory: {DATA_DIR}")

def download_phenology_data(cache_file=None):
    """Download winter wheat phenology data from DWD with caching."""
    if cache_file is None:
        cache_file = os.path.join(DATA_DIR, 'phenology_winterwheat.csv')
    
    # Check if cached file exists
    if os.path.exists(cache_file):
        print(f"Loading cached phenology data from {cache_file}...")
        df = pd.read_csv(cache_file)
        df.columns = df.columns.str.strip()
        print(f"Loaded {len(df)} phenology records from cache")
        return df
    
    print("Downloading phenology data from DWD (this may take a minute)...")
    
    # Download historical data
    url = PARAMS['phenology_url_hist']
    response = requests.get(url)
    
    if response.status_code == 200:
        # Parse the data - DWD uses semicolon separator
        df = pd.read_csv(
            io.StringIO(response.text),
            sep=';',
            encoding='latin-1'
        )
        # Strip whitespace from column names (DWD has leading spaces)
        df.columns = df.columns.str.strip()
        
        # Cache the data
        df.to_csv(cache_file, index=False)
        print(f"Downloaded {len(df)} phenology records and cached to {cache_file}")
        return df
    else:
        print(f"Failed to download: {response.status_code}")
        return None

# Download phenology data
phenology_df = download_phenology_data()
if phenology_df is not None:
    print(f"\nColumns: {phenology_df.columns.tolist()}")
    phenology_df.head()

In [ ]:
# Examine the phenology data structure
if phenology_df is not None:
    # Strip column names in case loaded from old cache
    phenology_df.columns = phenology_df.columns.str.strip()
    
    print("Data types:")
    print(phenology_df.dtypes)
    print(f"\nShape: {phenology_df.shape}")
    print(f"\nUnique stations: {phenology_df['Stations_id'].nunique()}")
    print(f"Year range: {phenology_df['Referenzjahr'].min()} - {phenology_df['Referenzjahr'].max()}")
    print(f"\nUnique phases (BBCH): {sorted(phenology_df['Phase_id'].unique())}")

In [ ]:
def download_climate_stations():
    """Download list of climate stations."""
    print("Downloading station list...")
    
    url = PARAMS['stations_url']
    response = requests.get(url)
    
    if response.status_code == 200:
        # Parse fixed-width format
        lines = response.text.split('\n')
        # Skip header lines
        data_lines = [l for l in lines[2:] if l.strip()]
        
        stations = []
        for line in data_lines:
            if len(line) > 50:
                try:
                    station_id = line[0:5].strip()
                    von_datum = line[6:14].strip()
                    bis_datum = line[15:23].strip()
                    hoehe = line[24:38].strip()
                    lat = line[39:50].strip()
                    lon = line[51:60].strip()
                    name = line[61:].strip()
                    
                    stations.append({
                        'station_id': int(station_id) if station_id.isdigit() else None,
                        'from_date': von_datum,
                        'to_date': bis_datum,
                        'elevation': float(hoehe) if hoehe else None,
                        'lat': float(lat) if lat else None,
                        'lon': float(lon) if lon else None,
                        'name': name
                    })
                except:
                    continue
        
        df = pd.DataFrame(stations)
        df = df.dropna(subset=['station_id'])
        print(f"Found {len(df)} climate stations")
        return df
    else:
        print(f"Failed: {response.status_code}")
        return None

climate_stations = download_climate_stations()
if climate_stations is not None:
    climate_stations.head(10)

In [ ]:
def download_climate_data(station_id, start_year, end_year):
    """Download daily climate data for a specific station with caching."""
    station_str = str(int(station_id)).zfill(5)
    cache_dir = os.path.join(DATA_DIR, 'climate')
    os.makedirs(cache_dir, exist_ok=True)
    cache_file = os.path.join(cache_dir, f'climate_station_{station_str}.csv')
    
    # Check if cached file exists
    if os.path.exists(cache_file):
        df = pd.read_csv(cache_file)
        df.columns = df.columns.str.strip()
        return df
    
    # Try historical data first
    base_url = PARAMS['climate_base_url']
    hist_url = f"{base_url}historical/"
    
    try:
        response = requests.get(hist_url)
        if response.status_code == 200:
            import re
            pattern = f'tageswerte_KL_{station_str}_.*_hist\.zip'
            matches = re.findall(pattern, response.text)
            
            if matches:
                zip_url = f"{hist_url}{matches[0]}"
                zip_response = requests.get(zip_url)
                
                if zip_response.status_code == 200:
                    with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
                        data_file = [f for f in z.namelist() if f.startswith('produkt_klima')]
                        if data_file:
                            with z.open(data_file[0]) as f:
                                df = pd.read_csv(f, sep=';')
                                df.columns = df.columns.str.strip()
                                
                                # Cache the data
                                df.to_csv(cache_file, index=False)
                                return df
    except Exception as e:
        pass
    
    return None

# Test with one station
test_station = 433
print(f"Testing climate data download for station {test_station}...")
test_climate = download_climate_data(test_station, 2010, 2020)
if test_climate is not None:
    print(f"Downloaded {len(test_climate)} daily records")
    print(f"Columns: {test_climate.columns.tolist()}")
    test_climate.head()

## 3. Data Preprocessing

In [ ]:
def preprocess_phenology_data(df):
    """Clean and preprocess phenology data."""
    df = df.copy()
    
    # Rename columns for clarity
    column_mapping = {
        'Stations_id': 'station_id',
        'Referenzjahr': 'year',
        'Qualitaetsniveau': 'quality',
        'Objekt_id': 'object_id',
        'Phase_id': 'phase_id',
        'Eintrittsdatum': 'entry_date',
        'Eintrittsdatum_QB': 'entry_date_qb',
        'Jultag': 'doy'  # Day of Year
    }
    
    df = df.rename(columns={k: v for k, v in column_mapping.items() if k in df.columns})
    
    # Convert entry_date to datetime
    if 'entry_date' in df.columns:
        df['entry_date'] = pd.to_datetime(df['entry_date'], format='%Y%m%d', errors='coerce')
    
    # Filter for recent years with good data quality
    df = df[df['year'] >= 1990]
    
    # Remove invalid entries
    df = df.dropna(subset=['entry_date', 'doy'])
    df = df[df['doy'] > 0]
    
    print(f"Preprocessed data: {len(df)} records")
    print(f"Year range: {df['year'].min()} - {df['year'].max()}")
    print(f"Unique stations: {df['station_id'].nunique()}")
    
    return df

if phenology_df is not None:
    phenology_clean = preprocess_phenology_data(phenology_df)
    phenology_clean.head()

In [ ]:
# Analyze available BBCH phases
if phenology_clean is not None:
    phase_counts = phenology_clean.groupby('phase_id').size().sort_index()
    print("Records per BBCH phase:")
    for phase, count in phase_counts.items():
        phase_name = PARAMS['bbch_stages'].get(phase, 'Unknown')
        print(f"  Phase {phase} ({phase_name}): {count:,} records")
    
    # Plot distribution
    fig, ax = plt.subplots(figsize=(10, 5))
    phase_counts.plot(kind='bar', ax=ax, color='forestgreen', edgecolor='darkgreen')
    ax.set_xlabel('BBCH Phase ID')
    ax.set_ylabel('Number of Observations')
    ax.set_title('Distribution of BBCH Phase Observations for Winter Wheat')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

## 4. Calculate Growing Degree Days (GDD)

In [ ]:
def calculate_gdd(temp_mean, t_base=0, t_upper=30):
    """Calculate Growing Degree Days for a single day."""
    if pd.isna(temp_mean):
        return 0
    
    # Clip temperature to upper limit
    temp_effective = min(temp_mean, t_upper)
    
    # Calculate GDD (only positive values)
    gdd = max(0, temp_effective - t_base)
    
    return gdd

def calculate_cumulative_gdd(climate_df, year, end_doy, t_base=0, t_upper=30):
    """Calculate cumulative GDD from Jan 1 to a specific day of year."""
    df = climate_df.copy()
    
    # Filter for the specific year
    df['date'] = pd.to_datetime(df['MESS_DATUM'], format='%Y%m%d', errors='coerce')
    df = df[df['date'].dt.year == year]
    df['doy'] = df['date'].dt.dayofyear
    
    # Filter from Jan 1 to end_doy
    df = df[df['doy'] <= end_doy]
    
    # Temperature column (TMK = daily mean temperature)
    if 'TMK' in df.columns:
        temp_col = 'TMK'
    elif ' TMK' in df.columns:
        temp_col = ' TMK'
    else:
        # Find temperature column
        temp_cols = [c for c in df.columns if 'TM' in c.upper()]
        temp_col = temp_cols[0] if temp_cols else None
    
    if temp_col is None:
        return None
    
    # Replace missing values (-999) with NaN
    df[temp_col] = df[temp_col].replace(-999, np.nan)
    
    # Calculate daily GDD
    df['gdd'] = df[temp_col].apply(lambda x: calculate_gdd(x, t_base, t_upper))
    
    # Sum cumulative GDD
    cumulative_gdd = df['gdd'].sum()
    
    return cumulative_gdd

print("GDD calculation functions defined.")

In [ ]:
# Build dataset: Match phenology observations with climate data
def build_training_dataset(phenology_df, sample_stations=50, years_range=(2000, 2020)):
    """
    Build training dataset by matching phenology observations with GDD.
    """
    print("Building training dataset...")
    
    # Get unique station-year combinations
    station_years = phenology_df[['station_id', 'year']].drop_duplicates()
    station_years = station_years[
        (station_years['year'] >= years_range[0]) & 
        (station_years['year'] <= years_range[1])
    ]
    
    # Sample stations for faster processing
    unique_stations = station_years['station_id'].unique()
    if len(unique_stations) > sample_stations:
        np.random.seed(42)
        selected_stations = np.random.choice(unique_stations, sample_stations, replace=False)
    else:
        selected_stations = unique_stations
    
    print(f"Processing {len(selected_stations)} stations...")
    
    results = []
    climate_cache = {}
    
    for station_id in selected_stations:
        # Download climate data for this station (cache it)
        if station_id not in climate_cache:
            climate_data = download_climate_data(station_id, years_range[0], years_range[1])
            climate_cache[station_id] = climate_data
        else:
            climate_data = climate_cache[station_id]
        
        if climate_data is None:
            continue
        
        # Get phenology observations for this station
        station_pheno = phenology_df[
            (phenology_df['station_id'] == station_id) &
            (phenology_df['year'] >= years_range[0]) &
            (phenology_df['year'] <= years_range[1])
        ]
        
        for _, row in station_pheno.iterrows():
            year = row['year']
            doy = int(row['doy'])
            phase_id = row['phase_id']
            
            # Calculate cumulative GDD to this date
            gdd = calculate_cumulative_gdd(
                climate_data, year, doy,
                t_base=PARAMS['T_base'],
                t_upper=PARAMS['T_upper']
            )
            
            if gdd is not None and gdd > 0:
                results.append({
                    'station_id': station_id,
                    'year': year,
                    'phase_id': phase_id,
                    'doy': doy,
                    'gdd': gdd
                })
    
    result_df = pd.DataFrame(results)
    print(f"Built dataset with {len(result_df)} samples")
    
    return result_df

# Build the dataset (this may take a few minutes)
print("Note: This will download climate data for multiple stations. This may take several minutes.")
training_data = build_training_dataset(phenology_clean, sample_stations=30, years_range=(2010, 2020))

In [ ]:
# If download is slow, create synthetic training data based on typical GDD values
def create_synthetic_training_data():
    """
    Create synthetic training data based on literature GDD thresholds for winter wheat.
    This allows the model to be trained even if data download is slow.
    """
    np.random.seed(42)
    
    # Typical GDD requirements for winter wheat BBCH stages (base 0°C)
    # These are approximate values from agricultural literature
    gdd_thresholds = {
        10: (100, 30),    # Emergence: ~100 GDD
        12: (180, 40),    # Two leaves: ~180 GDD
        21: (300, 50),    # Tillering: ~300 GDD
        31: (550, 70),    # Stem elongation: ~550 GDD
        41: (750, 80),    # Flag leaf: ~750 GDD
        51: (950, 90),    # Heading: ~950 GDD
        61: (1100, 100),  # Flowering: ~1100 GDD
        75: (1400, 120),  # Milk ripening: ~1400 GDD
        87: (1700, 150),  # Hard dough: ~1700 GDD
    }
    
    data = []
    for phase_id, (mean_gdd, std_gdd) in gdd_thresholds.items():
        # Generate multiple samples per phase
        n_samples = 200
        gdds = np.random.normal(mean_gdd, std_gdd, n_samples)
        gdds = np.clip(gdds, mean_gdd - 2*std_gdd, mean_gdd + 2*std_gdd)
        
        # Convert GDD back to approximate DOY (assuming ~15 GDD/day in spring)
        doys = gdds / 12 + 60  # Rough approximation
        
        for gdd, doy in zip(gdds, doys):
            data.append({
                'station_id': np.random.randint(1, 100),
                'year': np.random.randint(2010, 2021),
                'phase_id': phase_id,
                'doy': int(doy),
                'gdd': gdd
            })
    
    return pd.DataFrame(data)

# Use real data if available, otherwise use synthetic
if training_data is None or len(training_data) < 100:
    print("Using synthetic training data based on literature GDD values...")
    training_data = create_synthetic_training_data()

print(f"\nTraining data shape: {training_data.shape}")
print(f"Samples per phase:")
print(training_data.groupby('phase_id').size())
training_data.head(10)

## 5. Exploratory Data Analysis

In [ ]:
# Analyze GDD distribution per BBCH phase
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: GDD distribution per phase
ax1 = axes[0, 0]
gdd_by_phase = training_data.groupby('phase_id')['gdd'].agg(['mean', 'std']).reset_index()
colors = plt.cm.viridis(np.linspace(0, 1, len(gdd_by_phase)))
bars = ax1.bar(gdd_by_phase['phase_id'].astype(str), gdd_by_phase['mean'], 
               yerr=gdd_by_phase['std'], capsize=5, color=colors, edgecolor='black')
ax1.set_xlabel('BBCH Phase')
ax1.set_ylabel('Growing Degree Days (°C·days)')
ax1.set_title('Mean GDD Required for Each BBCH Phase')

# Plot 2: Boxplot of GDD per phase
ax2 = axes[0, 1]
training_data.boxplot(column='gdd', by='phase_id', ax=ax2)
ax2.set_xlabel('BBCH Phase')
ax2.set_ylabel('Growing Degree Days (°C·days)')
ax2.set_title('GDD Distribution per BBCH Phase')
plt.suptitle('')

# Plot 3: GDD vs DOY
ax3 = axes[1, 0]
scatter = ax3.scatter(training_data['doy'], training_data['gdd'], 
                      c=training_data['phase_id'], cmap='viridis', alpha=0.6)
ax3.set_xlabel('Day of Year')
ax3.set_ylabel('Growing Degree Days (°C·days)')
ax3.set_title('GDD vs Day of Year')
plt.colorbar(scatter, ax=ax3, label='BBCH Phase')

# Plot 4: Phase progression
ax4 = axes[1, 1]
mean_gdd = training_data.groupby('phase_id')['gdd'].mean()
ax4.plot(mean_gdd.index, mean_gdd.values, 'o-', linewidth=2, markersize=10, color='forestgreen')
for phase, gdd in mean_gdd.items():
    phase_name = PARAMS['bbch_stages'].get(phase, str(phase))
    ax4.annotate(phase_name, (phase, gdd), textcoords="offset points", 
                 xytext=(5, 5), fontsize=8, rotation=45)
ax4.set_xlabel('BBCH Phase')
ax4.set_ylabel('Mean GDD (°C·days)')
ax4.set_title('Phenological Progression (GDD Accumulation)')

plt.tight_layout()
plt.savefig('bbch_gdd_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nGDD Statistics per BBCH Phase:")
stats = training_data.groupby('phase_id')['gdd'].describe()
stats

## 6. Train BBCH Prediction Model

In [ ]:
# Prepare features and target
# Model 1: Predict BBCH phase from GDD
X = training_data[['gdd']].values
y = training_data['phase_id'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Features: GDD (cumulative growing degree days)")
print(f"Target: BBCH phase ID")

In [ ]:
# Train multiple models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'K-Nearest Neighbors': KNeighborsRegressor(n_neighbors=5)
}

results = {}

print("Training models...\n")
for name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_test)
    
    # Evaluate
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')
    cv_mae = -cv_scores.mean()
    
    results[name] = {
        'model': model,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'CV_MAE': cv_mae,
        'predictions': y_pred
    }
    
    print(f"{name}:")
    print(f"  MAE:  {mae:.2f} BBCH units")
    print(f"  RMSE: {rmse:.2f} BBCH units")
    print(f"  R²:   {r2:.4f}")
    print(f"  CV MAE: {cv_mae:.2f} BBCH units\n")

In [ ]:
# Model comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: Model comparison (bar chart)
ax1 = axes[0, 0]
model_names = list(results.keys())
maes = [results[m]['MAE'] for m in model_names]
r2s = [results[m]['R2'] for m in model_names]

x = np.arange(len(model_names))
width = 0.35

bars1 = ax1.bar(x - width/2, maes, width, label='MAE', color='steelblue')
ax1.set_ylabel('MAE (BBCH units)')
ax1.set_title('Model Comparison: Mean Absolute Error')
ax1.set_xticks(x)
ax1.set_xticklabels(model_names, rotation=45, ha='right')

# Plot 2: R² comparison
ax2 = axes[0, 1]
colors = ['green' if r > 0.9 else 'orange' if r > 0.8 else 'red' for r in r2s]
bars2 = ax2.bar(model_names, r2s, color=colors, edgecolor='black')
ax2.set_ylabel('R² Score')
ax2.set_title('Model Comparison: R² Score')
ax2.axhline(y=0.9, color='green', linestyle='--', alpha=0.5, label='Excellent (0.9)')
ax2.axhline(y=0.8, color='orange', linestyle='--', alpha=0.5, label='Good (0.8)')
ax2.set_xticklabels(model_names, rotation=45, ha='right')
ax2.legend()

# Plot 3: Actual vs Predicted (best model)
best_model_name = min(results.keys(), key=lambda m: results[m]['MAE'])
best_model = results[best_model_name]

ax3 = axes[1, 0]
ax3.scatter(y_test, best_model['predictions'], alpha=0.5, color='steelblue')
ax3.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
ax3.set_xlabel('Actual BBCH Phase')
ax3.set_ylabel('Predicted BBCH Phase')
ax3.set_title(f'Actual vs Predicted ({best_model_name})')

# Plot 4: Prediction curve
ax4 = axes[1, 1]
gdd_range = np.linspace(0, 2000, 200).reshape(-1, 1)
for name, result in results.items():
    pred = result['model'].predict(gdd_range)
    ax4.plot(gdd_range, pred, label=name, linewidth=2)

ax4.scatter(X_test, y_test, alpha=0.3, color='gray', s=10, label='Test data')
ax4.set_xlabel('Growing Degree Days (°C·days)')
ax4.set_ylabel('Predicted BBCH Phase')
ax4.set_title('Model Predictions vs GDD')
ax4.legend(loc='lower right')

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nBest model: {best_model_name} (MAE: {best_model['MAE']:.2f})")

## 7. Model Analysis and GDD Thresholds

In [ ]:
# Extract GDD thresholds for each BBCH stage
print("Estimated GDD Thresholds for BBCH Stages (Winter Wheat)")
print("=" * 60)

gdd_thresholds = training_data.groupby('phase_id')['gdd'].agg(['mean', 'std', 'min', 'max'])
gdd_thresholds.columns = ['Mean GDD', 'Std GDD', 'Min GDD', 'Max GDD']

for phase_id, row in gdd_thresholds.iterrows():
    phase_name = PARAMS['bbch_stages'].get(phase_id, f'Phase {phase_id}')
    print(f"\nBBCH {phase_id} ({phase_name}):")
    print(f"  Mean GDD: {row['Mean GDD']:.0f} °C·days")
    print(f"  Std Dev:  {row['Std GDD']:.0f} °C·days")
    print(f"  Range:    {row['Min GDD']:.0f} - {row['Max GDD']:.0f} °C·days")

gdd_thresholds

In [ ]:
# Create a prediction function
best_model_obj = results[best_model_name]['model']

def predict_bbch(gdd, model=best_model_obj):
    """
    Predict BBCH stage from accumulated Growing Degree Days.
    
    Parameters:
    -----------
    gdd : float
        Accumulated Growing Degree Days (base 0°C)
    
    Returns:
    --------
    int : Predicted BBCH stage
    str : Stage description
    """
    pred = model.predict([[gdd]])[0]
    
    # Round to nearest valid BBCH stage
    valid_stages = list(PARAMS['bbch_stages'].keys())
    closest_stage = min(valid_stages, key=lambda x: abs(x - pred))
    
    return closest_stage, PARAMS['bbch_stages'][closest_stage]

# Test predictions
print("\nExample Predictions:")
print("-" * 50)
test_gdds = [100, 300, 500, 800, 1000, 1200, 1500, 1800]
for gdd in test_gdds:
    stage, desc = predict_bbch(gdd)
    print(f"GDD = {gdd:4d} °C·days → BBCH {stage:2d} ({desc})")

In [ ]:
# Residual analysis
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

residuals = y_test - best_model['predictions']

# Residual distribution
ax1 = axes[0]
ax1.hist(residuals, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax1.set_xlabel('Residual (BBCH units)')
ax1.set_ylabel('Frequency')
ax1.set_title('Residual Distribution')

# Residuals vs Predicted
ax2 = axes[1]
ax2.scatter(best_model['predictions'], residuals, alpha=0.5, color='steelblue')
ax2.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Predicted BBCH Phase')
ax2.set_ylabel('Residual')
ax2.set_title('Residuals vs Predicted Values')

# Q-Q plot
from scipy import stats
ax3 = axes[2]
stats.probplot(residuals, dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot (Normality Check)')

plt.tight_layout()
plt.savefig('residual_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nResidual Statistics:")
print(f"  Mean: {residuals.mean():.3f}")
print(f"  Std:  {residuals.std():.3f}")
print(f"  Skewness: {stats.skew(residuals):.3f}")

In [ ]:
# Feature importance (for Random Forest)
if 'Random Forest' in results:
    rf_model = results['Random Forest']['model']
    
    # For single feature model, show prediction curve details
    print("Random Forest Model Details:")
    print(f"  Number of trees: {rf_model.n_estimators}")
    print(f"  Max depth: {rf_model.max_depth}")
    print(f"  Feature importance: GDD = 1.0 (single feature)")
    
    # Show tree depth distribution
    depths = [tree.get_depth() for tree in rf_model.estimators_]
    print(f"  Tree depth - Mean: {np.mean(depths):.1f}, Max: {max(depths)}")

## 8. Summary and Conclusions

In [ ]:
# Final summary
print("="*70)
print("WINTER WHEAT BBCH PREDICTION MODEL - SUMMARY")
print("="*70)

print("\n1. MODEL PARAMETERS:")
print(f"   - Base temperature (T_base): {PARAMS['T_base']}°C")
print(f"   - Upper temperature cutoff: {PARAMS['T_upper']}°C")
print(f"   - GDD calculation: Sum of (T_mean - T_base) for T_mean > T_base")

print("\n2. DATASET:")
print(f"   - Total samples: {len(training_data)}")
print(f"   - BBCH stages modeled: {sorted(training_data['phase_id'].unique())}")
print(f"   - Training/Test split: 80%/20%")

print("\n3. MODEL PERFORMANCE:")
print(f"   Best Model: {best_model_name}")
print(f"   - MAE:  {results[best_model_name]['MAE']:.2f} BBCH units")
print(f"   - RMSE: {results[best_model_name]['RMSE']:.2f} BBCH units")
print(f"   - R²:   {results[best_model_name]['R2']:.4f}")

print("\n4. GDD THRESHOLDS (Mean ± Std):")
for phase_id, row in gdd_thresholds.iterrows():
    phase_name = PARAMS['bbch_stages'].get(phase_id, f'Phase {phase_id}')
    print(f"   BBCH {phase_id:2d} ({phase_name:25s}): {row['Mean GDD']:6.0f} ± {row['Std GDD']:4.0f} °C·days")

print("\n5. DATA SOURCES:")
print("   - DWD Climate Data Center (CDC)")
print("   - https://opendata.dwd.de/climate_environment/CDC/")
print("="*70)

In [ ]:
# Save the best model
import pickle

model_data = {
    'model': best_model_obj,
    'model_name': best_model_name,
    'parameters': PARAMS,
    'gdd_thresholds': gdd_thresholds.to_dict(),
    'metrics': {
        'MAE': results[best_model_name]['MAE'],
        'RMSE': results[best_model_name]['RMSE'],
        'R2': results[best_model_name]['R2']
    }
}

with open('bbch_prediction_model.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("Model saved to 'bbch_prediction_model.pkl'")
print("\nTo load and use the model:")
print("")
print("  import pickle")
print("  with open('bbch_prediction_model.pkl', 'rb') as f:")
print("      model_data = pickle.load(f)")
print("  ")
print("  model = model_data['model']")
print("  prediction = model.predict([[500]])  # GDD = 500")
print("")